<div class="alert alert-block alert-info">
<center> <h1> Big Data Modeling and Management 2022 </h1> </center> <br>
<center> Assignment 2 - Answers for the Stakeholders (Queries) </center>

<hr>
<a class="anchor" id="group">
    
### Group
    
</a>

1. Gabriel Felipe Martins de Souza   	|   m20210598 <br>

2. Luiz Humberto Polaro Vizeu		    |   m20210554 <br>

3. Mohamed Shamsudeen                    |   m20210707 <br>

4. Rogerio Domingos Paulo	        	|   m20210597 <br>

5. Tongjiuzhou Liu                       |   m20211012 <br>


## 🚚 BDMM Second Homework Assignment 🚚 

_The Wide World Importers (WWI) is a wholesales novelty goods importer and distributor operating from the San Francisco bay area. In this assignment we will be working with their database._ 
You can get more information and details about the WWI database in the following link: https://docs.microsoft.com/en-us/sql/samples/wide-world-importers-what-is?view=sql-server-ver15

The focus of the second assignment is modelling. We will use the World Wide Importers database and convert it to a document-based database. To that end, we will be leveraging concepts like data denormalization, indices, and mongodb design patterns. 

More information on the extended datamodel to be found here: </br>  
https://docs.microsoft.com/en-us/sql/samples/wide-world-importers-oltp-database-catalog?view=sql-server-ver15

## Problem Description

Your team has just arrived at WWI (a leading company in logistics). Welcome!   <br>
Even though business is thriving, the IT department is going through a bad time.   <br>
Digitalization was never a priority for the company and now the company operational and analytical requirements are starting to grow beyond the capabilities of their existing data architecture.   <br>

WWI data are spread accross different systems, but we've already managed to pull them all into a mongo dump file. This data file is an exact dump of the SQL data so includes all the same structure, the SQL tables become collections and the rows become documents. This means all the original SQL keys are included in the data.<br>
Currently, the costs to develop the necessary queries to collect data to answer questions asked by the different departments are too high. <br>

Management concluded it is the right time to revise and revamp the data architecture, in order to speed up operations. 

In that context, your team was tasked with merging all the company data into a single and coherent Mongo database. <br>
It is expected that, with your solution, WWI will have a better understanding of their business and that the different departments will be able to obtain efficiently the answers they need.

The WWI team shared with you an ERD of their current datamodel:<br>
![datamodel](./WWI.png)

**Note** You can open the file WWI.png that is in the same directory as this notebook to see the above image in more detail and zoom in as you need.

Addtionally, the WWI team asked you the deliver the following outputs in **4 weeks**:
- Understand and model the database in MongoDB.
- Setup the database so that it is performs well for the queries they have provided. You should include reasoning in comments for the decisions you make on modelling the database.
- Answer the questions (queries) on the data provided.  
- Submit the results by following the instructions.  

With these deliveries, you will have created a prototype and allows the management to decide whether MongoDB is a good solution that meets their requirements.

### Design Requirements

Note that WWI has the following query requirements for the database.

1. The web team needs to know:  
    1. Which state province do we have the most suppliers in?  
    2. How many people have three or more `OtherLanguage`? 
    3. Top 10 most common `OtherLanguage` for people records. 
    4. How many customer records are valid after `November 2015`? 
    5. What percentage of people records don't have the UserPreferences field? 

2. The warehouse group needs to know:  
    1. What is the average difference in days between OrderDate and ExpectedDeliveryDate for orders sold by (`SalespersonPersonID`) person with the name `Jack Potter`?
    2. Which items get ordered the most in bulk (largest average quantity ordered)?  
    3. Which two items get ordered together the most?
    4. For each customer category which 3 items have the ordered the most?
    5. What is the current stock of each stockgroup?

3. The CFO needs to know:  
    1. What is the monthly total order count for each month?  
    2. How many orders are there from the customer `Tailspin Toys (Head Office)`?
    3. What are the average monthly sales prices of all goods sold? 
    4. In each state province what is the average customer credit limit?   
    5. What are the yearly expenditures with each supplier (per supplier name)?  

4. Partnerships needs to know:  
    1. What is the most common payment type?  
    2. What percentage of people have their `Title` as `Team Member`?
    3. Which supplier of the category `Novelty Goods Supplier` has the most transactions?  
    4. What is the highest `CommissionRate` that a person has?

5. The marketing team needs to know:  
    1. What is the name of the sales person with the largest sum of invoice values in 2013 (person whose customers paid the most money)?
    2. Who are the most common `PickedByPersonID` person names for orders done by customer `Adriana Pena`?
    3. How many people have in their name the string `Sara`?
    5. What are the top 10 most Common Names (Primary or Surnames) of people?

Transform the mongo dump file provided with this notebook and model a database following mongodb's best practices. You should adjust the data model to best fit the use cases provided above. Think about collections, embedding, linking, indexing, and the patterns learned in class. Provide justifications for each decision you make. What, if any, are the trade-off's or disadvantages of your approach.

Use MongoDB queries to answer the questions on your transformed database.

### Deliverables

1. Notebook with all DB creation operations and CRUD operations to create the data model. **Important** you should include in comments justification for your decisions on modelling the data.;
2. Second notebook with all required queries and answers for the questions, **Important** please indicate with comments the steps in the data model you took to optimise each query;


# Additional Information

## Groups  

Students should form groups of at least 4 and at most 5. <br>

## Submission  Deadline

The submission includes two notebooks with outputs (cells must be run). 
Please make sure to indicate:
1. group number,
2. group members with student names and numbers,
3. the name of the database that you created. <br>

Upload the notebook on moodle before **23:59 on June 22nd**

## Evaluation   

The second homework assignment counts 40% towards your final mark of the curricular unit. <br>
The assignment will be scored from 0 to 20. <br>

Each group submission will be evaluated on three components:
1. correctness of results;
2. simplicity and performance characteristics of the solution;
3. justification of decisions.

50% -  Database design  
50% -  Query results including performance

Please note that all code delivered in this assignment will go through plagiarism automated checks. <br>
Groups with high similarity levels in their code will undergo investigation.


In [27]:
#importing libraries to connect and run mongoDB methods
from pymongo import MongoClient
from bson.objectid import ObjectId
from pprint import pprint

In [28]:
#creating a connection with the database WideWorldImportes
host="localhost"
port="27017"
user="Group_GLMRT"
password="1024"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")
db = client.WideWorldImporters

# 1. The web team needs:
## A- Which state province do we have the most suppliers in?

1. We created the collection suppliers_cities_embed in the Modeling Notebook;
2. 1st query, selected the SupplierID and the Cities, and StateProvince from the list
3. 2nd query, grouped the results from qr1 in Stateprofince ids and summed up the number of suppliers
4. 3rd query, sorted the number of sup. in descending order
5. 

In [29]:
query_1 = { #selected the SupplierID and the Cities, and StateProvince from the Cities_list created in the DW notebook
    '$project': {'_id' : False,        
                'SupplierID' : '$SupplierID',
                'City' : '$Cities_list.CityName',
                'StateProvince' : '$Cities_list.StateProvince_list.StateProvinceName',}}

query_2 = { #grouped the results from query_1 in StateProvince and summed up the number of suppliers
    '$group':
        {'_id' : '$StateProvince', # will be groupped by this field
         'number_of_suppliers' : {'$sum' : 1}}} # type of accumulator

query_3 = { #sorted the number_of_suppliers in descending order
    '$sort':
        {'number_of_suppliers' : -1}} # -1 desc, 1 asc.

query_4 = {'$limit': 1}

pipeline = [query_1,query_2,query_3,query_4] 
    
result = list(db.suppliers_cities_embed.aggregate(pipeline))

result

[{'_id': [['California']], 'number_of_suppliers': 3}]

[{'_id': [['California']], 'number_of_suppliers': 3}]

## B- How many people have three or more OtherLanguage?

In [30]:
query_1 = {#selects OtherLanguages not equal to None
    '$match' : {
        'OtherLanguages': {'$ne': None}
    }    
}

query_2 = { # selects the OtherLanguages 
    '$project' : {
        '_id' : False,
        'OtherLanguages' : 1,
    }
}

query_3= {#finds OtherLanguages not equal to empty
    '$match' : {
        'OtherLanguages': {'$ne': '[]'}
    }    
}

query_4 = {#selects OtherLanguages, and uses trim function to remove strange character ']['
        '$project' : {
        '_id' : False,
        'OtherLanguages' : {'$trim' : {'input':'$OtherLanguages','chars':']['}}
    }
}

query_5 = { # selects OtherLanguages, and splits using comma symbol
    '$project' : {
        'OtherLanguages' : {'$split':['$OtherLanguages',',']}
    }
}

query_6 = { # size of OtherLanguages array
    '$project' : {
        '_id' : False,
        'OtherLanguages' : 1,
        'number_languages' : {'$size' : '$OtherLanguages'}
    }
}

query_7 = { #finds OtherLanguages where number_languages is greater than 3
    '$match' : {
        'OtherLanguages': {'$exists': True},
        'number_languages' : {'$gte' : 3}
    }    
}

query_8 = { # makes the count of the results from query_7
    '$count' : 'Number of people with 3 or more other languages'
}

pipeline = [query_1,query_2,query_3,query_4,query_5,query_6,query_7,query_8]

r = db.people.aggregate(pipeline)

result = list(r)

result

[{'Number of people with 3 or more other languages': 4}]

[{'Number of people with 3 or more other languages': 4}]

## C- Top 10 most common OtherLanguage for people records.

In [31]:
query_1 = {#finds OtherLanguages when it is not equal to None
    '$match' : {
        'OtherLanguages': {'$ne': None}
    }    
}

query_2 = { #  selects the OtherLanguages 
    '$project' : {
        '_id' : False,
        'OtherLanguages' : 1,
    }
}

query_3= { #finds OtherLanguages not equal to empty
    '$match' : {
        'OtherLanguages': {'$ne': '[]'}
    }    
}

query_4 = { #selects OtherLanguages, and uses trim function to remove strange character ']['
        '$project' : {
        '_id' : False,
        'OtherLanguages' : 1,
        'OtherLanguages' : {'$trim' : {'input':'$OtherLanguages','chars':']['}}
    }
}

query_5 = { # selects OtherLanguages, and splits using comma symbol
    '$project' : {
        'OtherLanguages' : 1,
        'OtherLanguages' : {'$split':['$OtherLanguages',',']}
    }
}

query_6 = { #Deconstruct OtherLanguages using unwind
    '$unwind' : '$OtherLanguages'
    }

query_7 = {
    '$group': {#Aggregates OtherLanguages using group by id and count the quantity of OtherLanguages
        '_id': '$OtherLanguages', 
        'count' : {'$sum' : 1}            
    }
}

query_8 = { #sorts the results in descending count
    '$sort': {
        'count': -1
    }
}


query_9 = { #shows the 10 first outputs
    '$limit': 10
}


pipeline = [query_1,query_2,query_3,query_4,query_5,query_6,query_7,query_8,query_9]

r = db.people.aggregate(pipeline)

result = list(r)

result

[{'_id': '"Greek"', 'count': 3},
 {'_id': '"Dutch"', 'count': 3},
 {'_id': '"Finnish"', 'count': 3},
 {'_id': '"Croatian"', 'count': 2},
 {'_id': '"Lithuanian"', 'count': 2},
 {'_id': '"Arabic"', 'count': 2},
 {'_id': '"Slovak"', 'count': 2},
 {'_id': '"Polish"', 'count': 2},
 {'_id': '"Romanian"', 'count': 2},
 {'_id': '"Japanese"', 'count': 1}]

[{'_id': '"Greek"', 'count': 3},
 {'_id': '"Dutch"', 'count': 3},
 {'_id': '"Finnish"', 'count': 3},
 {'_id': '"Croatian"', 'count': 2},
 {'_id': '"Lithuanian"', 'count': 2},
 {'_id': '"Arabic"', 'count': 2},
 {'_id': '"Slovak"', 'count': 2},
 {'_id': '"Polish"', 'count': 2},
 {'_id': '"Romanian"', 'count': 2},
 {'_id': '"Japanese"', 'count': 1}]

## D- How many customer records are valid after November 2015?

In [32]:
import datetime #imports datetime lib
#finds instances of customers where the ValidTo is greater than November 2015, by parsing the date with datetime
len(list(db.customers.find({'ValidTo':{'$gt':datetime.datetime(2015,11,30,23,59,59)}}, {'ValidTo':1, '_id':0})))

663

663

## E- What percentage of people records don't have the UserPreferences field?

In [33]:
#finds the length of UserPreferences not equal to None
NoneUserPreferences = len(list(db.people.find( { 'UserPreferences' : { '$ne': None } } )))
#finds the length of UserPreferences where exists equals False
NoExists = len(list(db.people.find({"UserPreferences": {"$exists": False}})))
#finds the total amount of people
TotalNoneUserPreferences = len(list(db.people.find()))
#retrieves the percentage in which UserPreferences field is None or Empty
Answer = (NoneUserPreferences + NoExists) / TotalNoneUserPreferences
#prints the output with the percentage format
print ("{0:.0%}".format(Answer))

16%
16%


# 2. The warehouse group needs to know:  

## A- What is the average difference in days between OrderDate and ExpectedDeliveryDate for orders sold by (`SalespersonPersonID`) person with the name `Jack Potter`?

In [34]:
query_1 = { #creates a new collection PersonInfo by joining the collection People with itself using PersonID
    "$lookup":
    {
       'from': 'people',
       'localField': 'SalespersonPersonID',
       'foreignField': 'PersonID',
       'as': 'PersonInfo'
     }
}

query_2 ={ #finds FullName inside PersonInfo, where it is equal to 'Jack Potter'
    '$match': {
        'PersonInfo.FullName' : {'$eq' : 'Jack Potter'}
    }
}

query_3 = { #selects SalespersonPersonID and the FullName, and the difference in days between the ExpectedDeliveryDate and OrderDate
    '$project' : {
        '_id' : False,
        'SalespersonPersonID' : 1,
        'PersonInfo.FullName' : 1,
        'days_in_between' : { '$subtract': [ '$ExpectedDeliveryDate', '$OrderDate' ] },
    }
}


query_4 = { #groups the output by FullName and performs an average agg function in the difference in days
    '$group': {
        '_id': {'FullName' : '$PersonInfo.FullName'}, 
        'average_days' : {'$avg' : '$days_in_between'}            
    }
}

query_5 = { #retrieves the result rounding to two decimals
    '$project' : {
        '_id' : False,
        'Average Days between Expected Delivery Date and Order Date is' : {'$round' : [{'$divide' : ['$average_days', 7387]}, 2]}
    }
}

pipeline = [query_1,query_2,query_3,query_4,query_5]


r=db.orders.aggregate(pipeline)

result = list(r)

result

[{'Average Days between Expected Delivery Date and Order Date is': 16948.2}]

[{'Average Days between Expected Delivery Date and Order Date is': 16948.2}]

## B- Which items get ordered the most in bulk (largest average quantity ordered)?  

In [35]:
query_1 = { #selects Description and Quantity from the collection
    '$project': {
        '_id' : 0,    
        'Description':1,
        'Quantity' : 1
    }
}

query_2 = { #groups by Description, and performs aggregation functions average, and sum over the Quantity
    '$group':{
        '_id' : '$Description', 
        'average_quantity' : {'$avg' : '$Quantity'},
        'total_quantity': { '$sum': '$Quantity' }
    }
} 

query_3 = { #rename the '_id' to 'items_name', and selects the average and total quantities
    '$project':{
        '_id':0,
        'items_name':'$_id',
        'average_quantity':1,
        'total_quantity':1       
    }
}

query_4 = {'$sort':{'average_quantity' : -1}} #sort the results in descending order based on average_quantity

query_5 = {'$limit': 3} #shows the first three outputs

pipeline = [query_1,query_2,query_3,query_4,query_5]
    
result = list(db.orderlines.aggregate(pipeline))

result

[{'average_quantity': 199.35,
  'total_quantity': 207324,
  'items_name': 'Black and orange fragile despatch tape 48mmx75m'},
 {'average_quantity': 198.23950870010236,
  'total_quantity': 193680,
  'items_name': 'Black and orange fragile despatch tape 48mmx100m'},
 {'average_quantity': 145.26190476190476,
  'total_quantity': 158626,
  'items_name': 'Clear packaging tape 48mmx75m'}]

[{'average_quantity': 199.35,
  'total_quantity': 207324,
  'items_name': 'Black and orange fragile despatch tape 48mmx75m'},
 {'average_quantity': 198.23950870010236,
  'total_quantity': 193680,
  'items_name': 'Black and orange fragile despatch tape 48mmx100m'},
 {'average_quantity': 145.26190476190476,
  'total_quantity': 158626,
  'items_name': 'Clear packaging tape 48mmx75m'}]

## C- Which two items get ordered together the most?

In [36]:
db.orderlines.create_index( #created the index for OrderID within orderlines
    [('OrderID', 1)],
    name='OrderID', 
)


db.orderlines.create_index( #created the index for Description within orderlines
    [('Description', 1)],
    name='Description', 
)

'Description'

'Description'

In [37]:
query_1 =     {#Joined orderlines with itself using OrderID
        '$lookup': {
           "from": "orderlines",
           "localField": "OrderID",
           "foreignField": "OrderID",
           "as": "joint_orders"
        }
}

query_2 = {#Deconstruct joint_orders
    '$unwind': '$joint_orders'
}

query_3 = {#Deconstruct the Description in joint_orders
    '$unwind': '$joint_orders.Description'
}
    
query_4 = {#selected the original Description and comparing with the Description from joint_orders
    '$project':{
        '_id':False,
        'order_items1':'$Description',
        'order_items2':'$joint_orders.Description',
        'order_together': {"$eq":["$Description","$joint_orders.Description"]}
    }
}

query_5 = {#Only kept the items that are not the same
    '$match':{"order_together": False}}

query_6 = {#Grouped in order_items1 and order_items2, then applied a distinct count on the number of instances
    '$group':{
        '_id': {'order_items1':'$order_items1','order_items2':'$order_items2'},
        'count': { '$sum': 1 }
    }
}

query_7 = {'$sort':{'count':-1}} #sorted the results in descending order

query_8 = {'$limit': 5} #limited 5 results in the output

pipeline = [query_1, query_2, query_3, query_4, query_5, query_6, query_7, query_8]

r = db.orderlines.aggregate(pipeline)

result = list(r)

result

[{'_id': {'order_items1': 'Dinosaur battery-powered slippers (Green) S',
   'order_items2': 'Animal with big feet slippers (Brown) XL'},
  'count': 30},
 {'_id': {'order_items1': 'Developer joke mug - fun was unexpected at this time (Black)',
   'order_items2': 'Air cushion film 200mmx200mm 325m'},
  'count': 30},
 {'_id': {'order_items1': 'Animal with big feet slippers (Brown) XL',
   'order_items2': 'Dinosaur battery-powered slippers (Green) S'},
  'count': 30},
 {'_id': {'order_items1': 'Air cushion film 200mmx200mm 325m',
   'order_items2': 'Developer joke mug - fun was unexpected at this time (Black)'},
  'count': 30},
 {'_id': {'order_items1': 'Black and orange glass with care despatch tape  48mmx100m',
   'order_items2': 'Developer joke mug - (hip, hip, array) (White)'},
  'count': 29}]

[{'_id': {'order_items1': 'Dinosaur battery-powered slippers (Green) S',
   'order_items2': 'Animal with big feet slippers (Brown) XL'},
  'count': 30},
 {'_id': {'order_items1': 'Developer joke mug - fun was unexpected at this time (Black)',
   'order_items2': 'Air cushion film 200mmx200mm 325m'},
  'count': 30},
 {'_id': {'order_items1': 'Animal with big feet slippers (Brown) XL',
   'order_items2': 'Dinosaur battery-powered slippers (Green) S'},
  'count': 30},
 {'_id': {'order_items1': 'Air cushion film 200mmx200mm 325m',
   'order_items2': 'Developer joke mug - fun was unexpected at this time (Black)'},
  'count': 30},
 {'_id': {'order_items1': 'Black and orange glass with care despatch tape  48mmx100m',
   'order_items2': 'Developer joke mug - (hip, hip, array) (White)'},
  'count': 29}]

## D- For each customer category which 3 items have the ordered the most?

In [38]:
# accesses the collection created in the Modeling Notebook
collection = db.orderlines_orders_customers_CustomerCategory_embed

query_1 = { #deconstructs the Customer_list array
    '$unwind' : '$Customer_list'
}

query_2 = { #deconstructs the orders_list array
    '$unwind' : '$orders_list'
}


query_3 = { #selects the Description, Quantity and CustomerCategoryName from the collection
    '$project':{
        'Description':1,
        'Quantity':1,
        'CustomerCategoryName':'$Customer_list.CustomerCategory_list.CustomerCategoryName'
    }
}

query_4 = { #deconstructs the CustomerCategoryName array
    '$unwind' : '$CustomerCategoryName'
}

query_5 = { #groups the output by _id, CustomerCategoryName and Description, and performs summation in Quantity 
    '$group':{'_id' : {'CustomerCategoryName':'$CustomerCategoryName','Description':'$Description'},
              'Quantity': { '$sum': '$Quantity'} 
        
    }
}

query_6 = { #selects CustomerCategoryName, Description and Quantity
    '$project':{
        '_id':0,
        'CustomerCategoryName':'$_id.CustomerCategoryName',
        'Description':'$_id.Description',
        'Quantity':1
    }
}

query_ = { #groups by CustomerCategoryName and retrieves the top 3 Descriptions based on Quantity in descending mode
    '$group':{ 
        '_id': "$CustomerCategoryName", 
        'Description':{
            '$topN':{
                'output': [ "$Description","$Quantity" ],
                'sortBy': { "Quantity": -1 },
                'n': 3
            }
        }
    }
}


query_7 = { #sorts the output in descending mode
    '$sort': { 'CustomerCategoryName': 1, 'Quantity': -1 } }

query_8 = { #groups by CustomerCategoryName and appends the Description into the outputs for each _id
    '$group': {
        '_id': '$CustomerCategoryName', 
        'items': { '$push': '$Description' },
    }
}

query_9 = { #selects the resultants CustomerCategory and limits the number of outputs to 3 using $slice
    '$project': {
        '_id':0,
        'CustomerCategory':'$_id',
        'top_three_items': {
            '$slice': ['$items', 3]
        }
    }
}


pipeline = [query_1,query_2,query_3,query_4,query_5,query_6,query_7,query_8,query_9]

r = collection.aggregate(pipeline)

result = list(r)

result

[{'CustomerCategory': 'Corporate',
  'top_three_items': ['Black and orange fragile despatch tape 48mmx75m',
   'Red and white urgent despatch tape 48mmx75m',
   'Shipping carton (Brown) 305x305x305mm']},
 {'CustomerCategory': 'Computer Store',
  'top_three_items': ['Shipping carton (Brown) 356x356x279mm',
   'Black and orange fragile despatch tape 48mmx75m',
   'Black and yellow heavy despatch tape 48mmx100m']},
 {'CustomerCategory': 'Gift Store',
  'top_three_items': ['Black and orange fragile despatch tape 48mmx75m',
   'Black and orange fragile despatch tape 48mmx100m',
   'Express post box 5kg (White) 350x280x130mm']},
 {'CustomerCategory': 'Novelty Shop',
  'top_three_items': ['Black and orange fragile despatch tape 48mmx75m',
   'Black and orange fragile despatch tape 48mmx100m',
   'Clear packaging tape 48mmx75m']},
 {'CustomerCategory': 'Supermarket',
  'top_three_items': ['Black and orange fragile despatch tape 48mmx75m',
   'Clear packaging tape 48mmx75m',
   'Black and orang

[{'CustomerCategory': 'Corporate',
  'top_three_items': ['Black and orange fragile despatch tape 48mmx75m',
   'Red and white urgent despatch tape 48mmx75m',
   'Shipping carton (Brown) 305x305x305mm']},
 {'CustomerCategory': 'Computer Store',
  'top_three_items': ['Shipping carton (Brown) 356x356x279mm',
   'Black and orange fragile despatch tape 48mmx75m',
   'Black and yellow heavy despatch tape 48mmx100m']},
 {'CustomerCategory': 'Gift Store',
  'top_three_items': ['Black and orange fragile despatch tape 48mmx75m',
   'Black and orange fragile despatch tape 48mmx100m',
   'Express post box 5kg (White) 350x280x130mm']},
 {'CustomerCategory': 'Novelty Shop',
  'top_three_items': ['Black and orange fragile despatch tape 48mmx75m',
   'Black and orange fragile despatch tape 48mmx100m',
   'Clear packaging tape 48mmx75m']},
 {'CustomerCategory': 'Supermarket',
  'top_three_items': ['Black and orange fragile despatch tape 48mmx75m',
   'Clear packaging tape 48mmx75m',
   'Black and orang

## E- What is the current stock of each stockgroup?

In [39]:
query_1 = { #selects Quantity and StockGroupName from the collection StockGroupID_list
    '$project': {'_id' : False,        
                'Quantity' : 1,
                'StockGroupName' : '$StockGroupID_list.StockGroupName'}}

query_2 = { #groups by StockGroupName, and performs a Sum in Quantity
    '$group':
        {'_id' : '$StockGroupName',
         'Quantity' : {'$sum' : '$Quantity'}}}

query_3 = { #sorts the results based on Quantity in descending mode
    '$sort':
        {'Quantity' : -1}}


pipeline = [query_1,query_2,query_3]
    
result = list(db.stockitemstransactions_stockitemsstockgroups_stockgroups_embed.aggregate(pipeline))

result

[{'_id': ['Clothing', 'T-Shirts', 'Computing Novelties'],
  'Quantity': Decimal128('89339225.000')},
 {'_id': ['Packaging Materials'], 'Quantity': Decimal128('42788674.000')},
 {'_id': ['Toys'], 'Quantity': Decimal128('-17138.000')},
 {'_id': ['Novelty Items', 'Computing Novelties'],
  'Quantity': Decimal128('-56010.000')},
 {'_id': ['Novelty Items', 'Computing Novelties', 'USB Novelties'],
  'Quantity': Decimal128('-79483.000')},
 {'_id': ['Clothing'], 'Quantity': Decimal128('-91321.000')},
 {'_id': ['Novelty Items', 'Toys'], 'Quantity': Decimal128('-102909.000')},
 {'_id': ['Novelty Items'], 'Quantity': Decimal128('-122280.000')},
 {'_id': ['Novelty Items', 'Mugs', 'Computing Novelties'],
  'Quantity': Decimal128('-242577.000')},
 {'_id': ['Clothing', 'Furry Footwear'],
  'Quantity': Decimal128('-394460.000')},
 {'_id': ['Novelty Items', 'Clothing'], 'Quantity': Decimal128('-554041.000')}]

[{'_id': ['Clothing', 'T-Shirts', 'Computing Novelties'],
  'Quantity': Decimal128('89339225.000')},
 {'_id': ['Packaging Materials'], 'Quantity': Decimal128('42788674.000')},
 {'_id': ['Toys'], 'Quantity': Decimal128('-17138.000')},
 {'_id': ['Novelty Items', 'Computing Novelties'],
  'Quantity': Decimal128('-56010.000')},
 {'_id': ['Novelty Items', 'Computing Novelties', 'USB Novelties'],
  'Quantity': Decimal128('-79483.000')},
 {'_id': ['Clothing'], 'Quantity': Decimal128('-91321.000')},
 {'_id': ['Novelty Items', 'Toys'], 'Quantity': Decimal128('-102909.000')},
 {'_id': ['Novelty Items'], 'Quantity': Decimal128('-122280.000')},
 {'_id': ['Novelty Items', 'Mugs', 'Computing Novelties'],
  'Quantity': Decimal128('-242577.000')},
 {'_id': ['Clothing', 'Furry Footwear'],
  'Quantity': Decimal128('-394460.000')},
 {'_id': ['Novelty Items', 'Clothing'], 'Quantity': Decimal128('-554041.000')}]

# 3. The CFO needs to know:

## A- What is the monthly total order count for each month?

In [40]:
query_1 =  { #selects month, year from OrderDate, and counts each instance as one
    '$project' : { 
          'month' : { '$month' : "$OrderDate"}, 
          'year' : { '$year' :  "$OrderDate"},
          'count' : { '$sum': 1 }
      }}
query_2 = { #groups by month and year, renaming it as _id, then counts the instances    
    '$group': {
            '_id' : {'month' : "$month", 'year' :'$year' },
            'count': { '$sum': '$count'} }}

query_3 = { #sorts the results based on year and month in ascending mode
    '$sort': {'_id.year':1,'_id.month':1}}

pipeline = [query_1,query_2, query_3 ]


r = db.orders.aggregate(pipeline)

result = list(r)

result

[{'_id': {'month': 1, 'year': 2013}, 'count': 1674},
 {'_id': {'month': 2, 'year': 2013}, 'count': 1139},
 {'_id': {'month': 3, 'year': 2013}, 'count': 1683},
 {'_id': {'month': 4, 'year': 2013}, 'count': 1696},
 {'_id': {'month': 5, 'year': 2013}, 'count': 1808},
 {'_id': {'month': 6, 'year': 2013}, 'count': 1675},
 {'_id': {'month': 7, 'year': 2013}, 'count': 1886},
 {'_id': {'month': 8, 'year': 2013}, 'count': 1537},
 {'_id': {'month': 9, 'year': 2013}, 'count': 1617},
 {'_id': {'month': 10, 'year': 2013}, 'count': 1618},
 {'_id': {'month': 11, 'year': 2013}, 'count': 1552},
 {'_id': {'month': 12, 'year': 2013}, 'count': 1565},
 {'_id': {'month': 1, 'year': 2014}, 'count': 1791},
 {'_id': {'month': 2, 'year': 2014}, 'count': 1538},
 {'_id': {'month': 3, 'year': 2014}, 'count': 1586},
 {'_id': {'month': 4, 'year': 2014}, 'count': 1739},
 {'_id': {'month': 5, 'year': 2014}, 'count': 1908},
 {'_id': {'month': 6, 'year': 2014}, 'count': 1887},
 {'_id': {'month': 7, 'year': 2014}, 'count

[{'_id': {'month': 1, 'year': 2013}, 'count': 1674},
 {'_id': {'month': 2, 'year': 2013}, 'count': 1139},
 {'_id': {'month': 3, 'year': 2013}, 'count': 1683},
 {'_id': {'month': 4, 'year': 2013}, 'count': 1696},
 {'_id': {'month': 5, 'year': 2013}, 'count': 1808},
 {'_id': {'month': 6, 'year': 2013}, 'count': 1675},
 {'_id': {'month': 7, 'year': 2013}, 'count': 1886},
 {'_id': {'month': 8, 'year': 2013}, 'count': 1537},
 {'_id': {'month': 9, 'year': 2013}, 'count': 1617},
 {'_id': {'month': 10, 'year': 2013}, 'count': 1618},
 {'_id': {'month': 11, 'year': 2013}, 'count': 1552},
 {'_id': {'month': 12, 'year': 2013}, 'count': 1565},
 {'_id': {'month': 1, 'year': 2014}, 'count': 1791},
 {'_id': {'month': 2, 'year': 2014}, 'count': 1538},
 {'_id': {'month': 3, 'year': 2014}, 'count': 1586},
 {'_id': {'month': 4, 'year': 2014}, 'count': 1739},
 {'_id': {'month': 5, 'year': 2014}, 'count': 1908},
 {'_id': {'month': 6, 'year': 2014}, 'count': 1887},
 {'_id': {'month': 7, 'year': 2014}, 'count

## B- How many orders are there from the customer Tailspin Toys (Head Office)?

In [41]:
query_1 = { #finds the CustomerName correspondent to "Tailspin Toys (Head Office)"
    '$match': {'CustomerName' : "Tailspin Toys (Head Office)"}
}

query_2 = { #joins orders with itself using CustomerID and renames the collection to "order"
    "$lookup":
    {
       "from": "orders",
       "localField": "CustomerID",
       "foreignField": "CustomerID",
       "as": "order"
     }
}

query_3 = { #selects the Name and matches the elements with $size in order
    "$project":{
        "_id": 0,
        'Name': '$CustomerName',
        "No_Orders": {'$size':"$order"}
    }
}


pipeline = [query_1, query_2, query_3]

r = db.customers.aggregate(pipeline)

result = list(r)

result

[{'Name': 'Tailspin Toys (Head Office)', 'No_Orders': 129}]

[{'Name': 'Tailspin Toys (Head Office)', 'No_Orders': 129}]

## C- What are the average monthly sales prices of all goods sold?

In [42]:
# accesses the collection order_orderline_embed, created in the Modeling Notebook
collection = db.order_orderline_embed

query_1 = { #selects the sales_prices by multiplying Quantity and UnitPrice, and the OrderDate from the collection created
    '$project':{
        'sales_prices':{'$multiply':['$Quantity','$UnitPrice']},
        'OrderDate':'$orders_list.OrderDate'      
    }
}

query_2 = { #deconstruct the OrderDate
    '$unwind' : '$OrderDate'
    }

query_3 = { #selects the sales_prices, month and year based on OrderDate
    '$project':{
        'sales_prices':1,
        'month' : { '$month' : "$OrderDate"}, 
        'year' : { '$year' :  "$OrderDate"},     
    }
}

query_4 = { #groups by the month and year, and performs agg function 'Avg' on sales_prices
    '$group':{'_id' : {'month' : "$month", 'year' :'$year' },
            'sales_prices': { '$avg': '$sales_prices'} 
        
    }
}
query_5 = { #selects the year, month, and sales_prices
    '$project':{
        '_id':0,
        'year':'$_id.year',
        'month':'$_id.month',
        'sales_prices':1      
    }
}

query_6 = {  #sorts the results based on year and month in ascending mode
    '$sort': {'year':1,'month':1}}

pipeline = [query_1,query_2,query_3,query_4,query_5,query_6]

r = collection.aggregate(pipeline)

result = list(r)

result

[{'sales_prices': Decimal128('724.2648835447831850028403711418292'),
  'year': 2013,
  'month': 1},
 {'sales_prices': Decimal128('757.1879227053140096618357487922705'),
  'year': 2013,
  'month': 2},
 {'sales_prices': Decimal128('735.9580812766747077379847838188903'),
  'year': 2013,
  'month': 3},
 {'sales_prices': Decimal128('782.0304949190816710575837410613474'),
  'year': 2013,
  'month': 4},
 {'sales_prices': Decimal128('800.6370152658361115985260572030181'),
  'year': 2013,
  'month': 5},
 {'sales_prices': Decimal128('777.4632071937055076807793180966654'),
  'year': 2013,
  'month': 6},
 {'sales_prices': Decimal128('763.6943436227951153324287652645862'),
  'year': 2013,
  'month': 7},
 {'sales_prices': Decimal128('747.2962440340319568375181572940444'),
  'year': 2013,
  'month': 8},
 {'sales_prices': Decimal128('767.9943616395371641498333006471857'),
  'year': 2013,
  'month': 9},
 {'sales_prices': Decimal128('750.3137594275768710114097853413266'),
  'year': 2013,
  'month': 10},

[{'sales_prices': Decimal128('724.2648835447831850028403711418292'),
  'year': 2013,
  'month': 1},
 {'sales_prices': Decimal128('757.1879227053140096618357487922705'),
  'year': 2013,
  'month': 2},
 {'sales_prices': Decimal128('735.9580812766747077379847838188903'),
  'year': 2013,
  'month': 3},
 {'sales_prices': Decimal128('782.0304949190816710575837410613474'),
  'year': 2013,
  'month': 4},
 {'sales_prices': Decimal128('800.6370152658361115985260572030181'),
  'year': 2013,
  'month': 5},
 {'sales_prices': Decimal128('777.4632071937055076807793180966654'),
  'year': 2013,
  'month': 6},
 {'sales_prices': Decimal128('763.6943436227951153324287652645862'),
  'year': 2013,
  'month': 7},
 {'sales_prices': Decimal128('747.2962440340319568375181572940444'),
  'year': 2013,
  'month': 8},
 {'sales_prices': Decimal128('767.9943616395371641498333006471857'),
  'year': 2013,
  'month': 9},
 {'sales_prices': Decimal128('750.3137594275768710114097853413266'),
  'year': 2013,
  'month': 10},

## D- In each state province what is the average customer credit limit?

In [43]:
query_1 = { #selects CreditLimit and StateProvinceName from the collection created in the Modeling Notebook
    '$project':{
        '_id':0,
        'CreditLimit':1,
        'StateProvinceName':'$city_list.StateProvince_list.StateProvinceName'
    }
}

query_2 = { #groups by StateProvinceName and performs an Avg function in CreditLimit
    
    '$group': {
            '_id' : '$StateProvinceName',
            'CreditLimit_avg': { '$avg': '$CreditLimit' } }}

query_3 = { #limits one output per instance
    '$limit': 1}

pipeline = [query_1,query_2 ]


r = db.customers_city_state_embed.aggregate(pipeline)

result = list(r)

result

[{'_id': [['Tennessee']], 'CreditLimit_avg': Decimal128('3465.00')},
 {'_id': [['Utah']], 'CreditLimit_avg': Decimal128('3200.00')},
 {'_id': [['Florida']], 'CreditLimit_avg': Decimal128('2800.00')},
 {'_id': [['Oklahoma']], 'CreditLimit_avg': Decimal128('2806.00')},
 {'_id': [['New Mexico']], 'CreditLimit_avg': Decimal128('2741.00')},
 {'_id': [['Texas']], 'CreditLimit_avg': Decimal128('2312.50')},
 {'_id': [['West Virginia']],
  'CreditLimit_avg': Decimal128('2567.857142857142857142857142857143')},
 {'_id': [['Kentucky']], 'CreditLimit_avg': Decimal128('2374.00')},
 {'_id': [['California']],
  'CreditLimit_avg': Decimal128('2787.307692307692307692307692307692')},
 {'_id': [['Pennsylvania']],
  'CreditLimit_avg': Decimal128('2255.833333333333333333333333333333')},
 {'_id': [['Virginia']], 'CreditLimit_avg': Decimal128('2915.00')},
 {'_id': [['Massachusetts[E]']], 'CreditLimit_avg': Decimal128('2507.50')},
 {'_id': [['Louisiana']],
  'CreditLimit_avg': Decimal128('2372.0416666666666666

[{'_id': [['Tennessee']], 'CreditLimit_avg': Decimal128('3465.00')},
 {'_id': [['Utah']], 'CreditLimit_avg': Decimal128('3200.00')},
 {'_id': [['Florida']], 'CreditLimit_avg': Decimal128('2800.00')},
 {'_id': [['Oklahoma']], 'CreditLimit_avg': Decimal128('2806.00')},
 {'_id': [['New Mexico']], 'CreditLimit_avg': Decimal128('2741.00')},
 {'_id': [['Texas']], 'CreditLimit_avg': Decimal128('2312.50')},
 {'_id': [['West Virginia']],
  'CreditLimit_avg': Decimal128('2567.857142857142857142857142857143')},
 {'_id': [['Kentucky']], 'CreditLimit_avg': Decimal128('2374.00')},
 {'_id': [['California']],
  'CreditLimit_avg': Decimal128('2787.307692307692307692307692307692')},
 {'_id': [['Pennsylvania']],
  'CreditLimit_avg': Decimal128('2255.833333333333333333333333333333')},
 {'_id': [['Virginia']], 'CreditLimit_avg': Decimal128('2915.00')},
 {'_id': [['Massachusetts[E]']], 'CreditLimit_avg': Decimal128('2507.50')},
 {'_id': [['Louisiana']],
  'CreditLimit_avg': Decimal128('2372.0416666666666666

## E- What are the yearly expenditures with each supplier (per supplier name)?

In [44]:
query_1 = { #selects the TransactionAmount, year from TransactionDate, and SupplierName from the collection created
    '$project':{
        'TransactionAmount':1,
        'year' : { '$year' :  "$TransactionDate"},
        'SupplierName':'$suppliers_list.SupplierName'        
    }
}

query_2 = { #groups the outputs by SupplierName and year, and performs a summation in TransactionAmount, named yearly_expenditures   
    '$group': {
            '_id' : {'SupplierName' : "$SupplierName", 'year' :'$year' },
            'yearly_expenditures': { '$sum': '$TransactionAmount'} }}

query_3 = { #retrieves SupplierName, year, and yearly_expenditures
    '$project':{ 
        '_id':0,
        'SupplierName':'$_id.SupplierName',
        'year' : '$_id.year',
        'yearly_expenditures':1       
    }
}

query_4 = { #sorts the results in year and SupplierName in ascending order
    '$sort': {'year':1,'SupplierName':1,}}


pipeline = [query_1,query_2,query_3,query_4]

r = db.suppliertransactions_suppliers_embed.aggregate(pipeline)

result = list(r)

result

[{'yearly_expenditures': Decimal128('0.00'),
  'SupplierName': ['Contoso, Ltd.'],
  'year': 2013},
 {'yearly_expenditures': Decimal128('485380.50'),
  'SupplierName': ['Fabrikam, Inc.'],
  'year': 2013},
 {'yearly_expenditures': Decimal128('0.00'),
  'SupplierName': ['Graphic Design Institute'],
  'year': 2013},
 {'yearly_expenditures': Decimal128('119343.32'),
  'SupplierName': ['Litware, Inc.'],
  'year': 2013},
 {'yearly_expenditures': Decimal128('0.00'),
  'SupplierName': ['Northwind Electric Cars'],
  'year': 2013},
 {'yearly_expenditures': Decimal128('0.00'),
  'SupplierName': ['The Phone Company'],
  'year': 2013},
 {'yearly_expenditures': Decimal128('1510955.10'),
  'SupplierName': ['Fabrikam, Inc.'],
  'year': 2014},
 {'yearly_expenditures': Decimal128('686937.21'),
  'SupplierName': ['Litware, Inc.'],
  'year': 2014},
 {'yearly_expenditures': Decimal128('2837528.40'),
  'SupplierName': ['Fabrikam, Inc.'],
  'year': 2015},
 {'yearly_expenditures': Decimal128('1353734.69'),
  '

[{'yearly_expenditures': Decimal128('0.00'),
  'SupplierName': ['Contoso, Ltd.'],
  'year': 2013},
 {'yearly_expenditures': Decimal128('485380.50'),
  'SupplierName': ['Fabrikam, Inc.'],
  'year': 2013},
 {'yearly_expenditures': Decimal128('0.00'),
  'SupplierName': ['Graphic Design Institute'],
  'year': 2013},
 {'yearly_expenditures': Decimal128('119343.32'),
  'SupplierName': ['Litware, Inc.'],
  'year': 2013},
 {'yearly_expenditures': Decimal128('0.00'),
  'SupplierName': ['Northwind Electric Cars'],
  'year': 2013},
 {'yearly_expenditures': Decimal128('0.00'),
  'SupplierName': ['The Phone Company'],
  'year': 2013},
 {'yearly_expenditures': Decimal128('1510955.10'),
  'SupplierName': ['Fabrikam, Inc.'],
  'year': 2014},
 {'yearly_expenditures': Decimal128('686937.21'),
  'SupplierName': ['Litware, Inc.'],
  'year': 2014},
 {'yearly_expenditures': Decimal128('2837528.40'),
  'SupplierName': ['Fabrikam, Inc.'],
  'year': 2015},
 {'yearly_expenditures': Decimal128('1353734.69'),
  '

# 4. Partnerships needs to know:

## A- What is the most common payment type?

In [45]:
query_1 = { #selects _id, SupplierTransactionID, and PaymentMethodName from the collection paymentmethods_list
    '$project':{
        '_id':1,
        'SupplierTransactionID':1,
        'PaymentMethodName':'$paymentmethods_list.PaymentMethodName'
    }
}

query_2 = { #groups the outputs by PaymentMethodName, makes a summation of the instances and stores in Number_of_Method
    '$group': {
            '_id' : '$PaymentMethodName',
            'Number_of_Method': { '$sum':1 } }}


pipeline = [query_1,query_2 ]


r = db.suppliertransactions_paymentmethods_embed.aggregate(pipeline)

result = list(r)

result

[{'_id': ['EFT'], 'Number_of_Method': 2438}]

[{'_id': ['EFT'], 'Number_of_Method': 2438}]

## B- What percentage of people have their Title as Team Member?

In [46]:
#finds the instances on which CustomerFields is equal to "Team Member" and retrieves the percentage by dividing for the total number of people
result = len(list(db.people.find({'CustomFields' : { '$regex':'"Team Member"' }})))/len(list(db.people.find()))
#converts the results to the percentage format
print ("{0:.2%}".format(result))

1.17%
1.17%


## C- Which supplier of the category Novelty Goods Supplier has the most transactions?

In [47]:
query_1 = { #finds the SupplierCategoryName that matches to Novelty Goods Supplier, using the collection created in the Modeling notebook
    '$match': {'suppliers_list.suppliercategories_list.SupplierCategoryName' :{'$eq' : 'Novelty Goods Supplier'}}
}

query_2 = { #selects the TransactionAmount and SupplierName
    '$project':{
        '_id':0,
        'TransactionAmount':1,
        'SupplierName':'$suppliers_list.SupplierName'
    }
}

query_3 = { #groups by SupplierName and counts the number of instances
    '$group': {
        '_id' : '$SupplierName',
        'count': { '$sum': 1 } 
    }
}

query_4 = { #sorts the results based on count in descending order
    '$sort': {'count':-1}}

query_5 = { #limits to one output
    '$limit':1}

pipeline = [query_1,query_2,query_3,query_4,query_5]


r = db.suppliertransactions_suppliers_suppliercategories_embed.aggregate(pipeline)

result = list(r)

result

[{'_id': ['Graphic Design Institute'], 'count': 16}]

[{'_id': ['Graphic Design Institute'], 'count': 16}]

## D- What is the highest CommissionRate that a person has?

In [48]:
query_1 = { #finds the CustomFields that matches with CommissionRate using $regex
    '$match': {'CustomFields' :{ '$regex':'"CommissionRate"' }}
}

query_2 = { #selects CustomFields, and retrieves the ComissionRate using the iterate functions for string and substrings $strLenCP and $substrCP
    '$project':{
        '_id':0,
        'CustomFields':1,
        "CommissionRate": { 
            '$substrCP': [
                "$CustomFields",{ '$subtract': [ { '$strLenCP': "$CustomFields" }, 6] },4]
        }
    }
}

query_3 = { #selects all the outputs, but CustomFields
    '$project':{
        'CustomFields':0}}

query_4 = { #sorts the results based in ComissionRate in descending Order
    '$sort': {'CommissionRate':-1}}

query_5 = { #shows only the first output
    '$limit':1}


pipeline = [query_1,query_2,query_3,query_4,query_5]


r = db.people.aggregate(pipeline)

result = list(r)

result

[{'CommissionRate': '4.55'}]

[{'CommissionRate': '4.55'}]

# 5. The marketing team needs to know:

## A- What is the name of the sales person with the largest sum of invoice values in 2013 (person whose customers paid the most money)?

In [49]:
# accesses the collection invoicelines_invoices_people_embed, created in the Modeling notebook
collection = db.invoicelines_invoices_people_embed


query_1 = { #selects FullName, invoice_values, invoiceDate from the collection
    '$project':{
        'FullName':'$invoice_list.people_list.FullName',
        'invoice_values':{'$multiply':['$Quantity','$UnitPrice']},
        'InvoiceDate':'$invoice_list.InvoiceDate',
    }
}

query_2 = { #deconstructs the FullName
    '$unwind' : '$FullName'
}

query_3 = {  #deconstructs the FullName @is this one necessary??
    '$unwind' : '$FullName'
}

query_4 = { #deconstructs the InvoiceDate
    '$unwind' : '$InvoiceDate'
}

query_5= {
    '$project':{  #selects invoice_values, FullName, and year from InvoiceDate
        'invoice_values':1,
        'FullName':1,
        'year' : { '$year' :  "$InvoiceDate"}  
    }
}

query_6 = { #finds instances that corresponds to year 2013
    '$match' : {
        'year': 2013
    }    
}

query_7 = { #groups the outputs by FullName, and performs a summation on invoice_values
    '$group':{'_id' : '$FullName',
              'invoice_values': { '$sum': '$invoice_values'} 
        
    }
}

query_8 = { #sorts the outputs based on invoice_values in descending order
    '$sort': {'invoice_values':-1}}

query_9 = { #shows only one output
    '$limit':1}

pipeline = [query_1,query_2,query_3,query_4,query_5,query_6,query_7,query_8,query_9]

r = collection.aggregate(pipeline)

result = list(r)

result

[{'_id': 'Hudson Onslow', 'invoice_values': Decimal128('4864279.75')}]

[{'_id': 'Hudson Onslow', 'invoice_values': Decimal128('4864279.75')}]

## B- Who are the most common PickedByPersonID person names for orders done by customer Adriana Pena?

In [50]:
query_1 = { #finds the instances where CustomerName is equal to 'Adriana Pena', and PersonalInfo.FullName exists and is not equal to empty
    '$match' : {
        'CustomerInfo.CustomerName' : {'$eq' : 'Adriana Pena'},
        'PersonInfo.FullName' : {'$exists': True, '$ne': []} 
    }
}

query_2 = { #selects CustomerID and FullName as 1
    '$project' : {
        '_id' : False,
        'CustomerID' : 1,
        'PersonInfo.FullName' : 1
    }
}


query_3 = { #groups by FullName, and performs a summation in 1 (FullName)
    '$group': {
        '_id': {'FullName' : '$PersonInfo.FullName'}, 
        'count' : {'$sum': 1}           
    }
}


query_4 = { #sorts the results based on count in descending order
    '$sort': {
        'count': -1
    }
}


query_5 = { #shows only the first three results
    '$limit': 3
}


query_6 = { #selects FullName and the Count
    '$project' : {
        '_id' : False,
        'FullName' : '$_id.FullName',
        'Count' : '$count'
    }
}

pipeline = [query_1,query_2,query_3,query_4,query_5,query_6]


r = db.orders_people_customers_embed.aggregate(pipeline)

result = list(r)

result

[{'FullName': ['Anthony Grosse'], 'Count': 3},
 {'FullName': ['Piper Koch'], 'Count': 3},
 {'FullName': ['Katie Darwin'], 'Count': 3}]

[{'FullName': ['Anthony Grosse'], 'Count': 3},
 {'FullName': ['Piper Koch'], 'Count': 3},
 {'FullName': ['Katie Darwin'], 'Count': 3}]

## C- How many people have in their name the string Sara?

In [51]:
#finds people with the FullName equal to 'Sara' using $regex, and retrieves the amount by using the len of the list
len(list(db.people.find({"FullName": {'$regex': 'Sara'} },{"FullName":1,"_id":1})))

5

5

## D- What are the top 10 most Common Names (Primary or Surnames) of people?

In [52]:
query_1 = { #selects the Fullname splited by its spaces " ", and rename to name
    '$project': {
        '_id' : False,
        'name' : { '$split': ["$FullName" , " "]}}
}

query_2 = { #deconstructs name
    '$unwind' : "$name"
}

query_3 = { #finds the instances where name corresponds to ".+"
    "$match": {
        "name": {"$regex": ".+"}
    }
}

query_4 = { #groups the outputs by name and count the number of instances
    '$group': {
        '_id': {'name' : '$name'},
        'count' : {'$sum' : 1}
    }
}

query_5 = { #sorts the results based on Count in descending order
    '$sort' : { 'count' : -1 }
}

query_6 = { #limits the output for the first 10 results
    '$limit': 10
}

query_7 = { #selects the Name and the Count
    '$project': {
        '_id' : False,
        'name' : '$_id.name',
        'count': 1}
}

pipeline = [query_1,query_2,query_3,query_4,query_5,query_6,query_7]

r = db.people.aggregate(pipeline)

result = list(r)

result

[{'count': 8, 'name': 'Bose'},
 {'count': 7, 'name': 'Ganguly'},
 {'count': 6, 'name': 'Thakur'},
 {'count': 6, 'name': 'Roman'},
 {'count': 5, 'name': 'Dhanishta'},
 {'count': 5, 'name': 'Van'},
 {'count': 5, 'name': 'PrabhupÄ\x81da'},
 {'count': 5, 'name': 'De'},
 {'count': 5, 'name': 'David'},
 {'count': 5, 'name': 'Mukherjee'}]

[{'count': 8, 'name': 'Bose'},
 {'count': 7, 'name': 'Ganguly'},
 {'count': 6, 'name': 'Thakur'},
 {'count': 6, 'name': 'Roman'},
 {'count': 5, 'name': 'Dhanishta'},
 {'count': 5, 'name': 'Van'},
 {'count': 5, 'name': 'PrabhupÄ\x81da'},
 {'count': 5, 'name': 'De'},
 {'count': 5, 'name': 'David'},
 {'count': 5, 'name': 'Mukherjee'}]